In [ ]:
import os
os.chdir('d:/future/Index_Future_Prediction')

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.optim import lr_scheduler, Adam, AdamW
from torch.utils.data import TensorDataset, DataLoader

import optuna
from utils import *
from modules.truncate import SequenceTruncate

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)
warnings.filterwarnings("ignore", category=UserWarning)


In [ ]:
class Baseline_LSTM(nn.Module):
    """循环神经网络模型"""
    def __init__(self, input_size, hidden_size, num_layers, dropout, **kwargs):
        super(Baseline_LSTM, self).__init__(**kwargs)
        self.device = 'cuda:0'
        self.hidden_size = hidden_size
        self.truncate = SequenceTruncate(dropout)
        self.process = nn.LSTM(
            input_size = input_size,
            hidden_size = hidden_size,
            num_layers = num_layers,
            dropout = dropout,
            batch_first = True,
            # nonlinearity='relu',
        )
        self.output = nn.Sequential(
            nn.Dropout(dropout),
            HybridDecoder(dim_state = hidden_size, init_prob = [0.0,0.5,0.0])
        )
        

    def forward(self, x):
        #为了提升模型的泛化能力，我们每次输入都随机舍弃一部分前端的序列
        front_shape = x.shape[:-2]
        x = self.truncate(x)
        x = torch.flatten(x, 0, -3)
        x = self.process(x)[0][:,-1,:]
        x = x.reshape(*front_shape, self.hidden_size)
        return self.output(x)

In [ ]:
def objective(trial):
    weight_decay = trial.suggest_float("weight_decay", 1e-6, 1e-4, log=True) # 对数均匀分布
    seq_len = trial.suggest_int("seq_len", 30, 40)


    # 提取数据
    feature_columns = ['inday_chg_open','inday_chg_high','inday_chg_low','inday_chg_close','inday_chg_amplitude', 'ma_10','ma_26','ma_45','ma_90','ma_vol',]
    label_columns = ['label_return','down_prob','middle_prob','up_prob']
    assets_list = ['IH.CFX', 'IF.CFX', 'IC.CFX', 'AU.SHF', 'JM.DCE','RB.SHF','HC.SHF', 'I.DCE', 'M.DCE', 'CF.ZCE',]
    assets_list = ['IH.CFX', 'IF.CFX', 'IC.CFX',]
    feature = []
    label = []

    for asset_code in assets_list:
        data = pd.read_csv(f'data/{asset_code}.csv')
        feature.append(torch.tensor(data[feature_columns].values, dtype = torch.float32, device = 'cuda:0'))
        label.append(torch.tensor(data[label_columns].values, dtype = torch.float32, device = 'cuda:0'))

    # 加载数据
    feature = torch.stack(feature, dim = 1)
    label = torch.stack(label, dim = 1)
    print(feature.shape, label.shape)

    # 折叠时间步
    feature = feature.unfold(dimension = 0, size = seq_len, step = 1).transpose(2,3)
    label = label[seq_len-1:]
    print(feature.shape, label.shape)

    data = RandomLoader(feature, label)
    recorder = PredictionRecorder()
    animator = TrainMonitor(figsize=(12,6))

    result = np.zeros(shape = (10, len(assets_list), 4))
    precision_list = []

    result = []
    for i in range(10):
        j = 0
        train_loader, test_loader = data(batch_size=64, slice_size=[0.6, 0.1], balance=[True, False])
        
        animator.reset()
        loss_fn = HybridLoss(alpha = 0.02, delta = 1.3, show_loss = True) #控制损失在1：3左右
        model = Baseline_LSTM(input_size = 10, hidden_size = 5, num_layers = 2, dropout = 0.3).to('cuda:0')
        optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay = weight_decay)
        scheduler = lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.9)
        train = ModelTrain(model = model,
                    train_loader = train_loader,
                    test_loader = test_loader,
                    loss_fn = loss_fn,
                    optimizer = optimizer,
                    scheduler = scheduler,
                    recorder = recorder,
                    graph = animator,
                    )
        prediction, precision = train.epoch_train(epochs = 10, early_stop = 100)

        precision_list.append(precision)


    return np.mean(precision_list)/np.std(precision_list)

In [ ]:
# --- 运行优化 ---
if __name__ == "__main__":
    # 我们要最大化准确率
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=30)  # 运行30次试验

    print("优化完成!")
    print("最佳试验的编号: ", study.best_trial.number)
    print("最佳准确率: ", study.best_value)
    print("最佳超参数: ", study.best_params)

In [ ]:
# all_assets = pd.DataFrame({
#     'stage_1_prediction': np.mean(result, axis = 0)[:,0],
#     'stage_2_prediction': np.mean(result, axis = 0)[:,2],

#     'stage_1_precision': np.mean(result, axis = 0)[:,1],
#     'stage_2_precision': np.mean(result, axis = 0)[:,3],

#     'stage_1_precision_std': np.std(result, axis = 0)[:,1],
#     'stage_2_precision_std': np.std(result, axis = 0)[:,3],
# })
# all_assets.index = pd.Series(assets_list)
# for col in all_assets.columns:
#     all_assets[col] = all_assets[col].apply(lambda x: f"{x:.1%}")

# # 转换为Markdown
# markdown_table = all_assets.to_markdown(index=False)
# print(f'hidden_size: {hidden_size}, num_layers: {num_layers}, seq_len: {seq_len}')
# print(markdown_table)

优化完成!
最佳试验的编号:  15
最佳准确率:  1.7570846318543656
最佳超参数:  {'learning_rate': 0.0017696481376409101, 'weight_decay': 0.00036082540684983, 'dropout': 0.3203330257231537, 'batch_size': 64, 'seq_len': 40, 'hidden_size': 10, 'num_layers': 2}


最佳试验的编号:  3
最佳准确率:  1.2913507249916159
最佳超参数:  {'learning_rate': 0.0012439264519395815, 'weight_decay': 2.2229309086256647e-05, 'dropout': 0.3640695831669655, 'batch_size': 64, 'seq_len': 30, 'hidden_size': 5, 'num_layers': 2}

最佳试验的编号:  7
最佳准确率:  1.0722981008506158
最佳超参数:  {'weight_decay': 4.048437595935014e-06, 'seq_len': 38, 'hidden_size': 5, 'num_layers': 2}